<a href="https://colab.research.google.com/github/dheerajpreetsingh/APMLP/blob/main/APMLP_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
from PIL import Image
from IPython.display import display
from matplotlib import pyplot as plt
import statistics
import math
import dlib
import cv2
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import tensorflow as tf

In [ ]:
from google.colab import drive

drive.mount('/content/drive')



Mounted at /content/drive


In [3]:
fem1=pd.read_csv("/content/drive/MyDrive/APMLP/Rating_Female.csv", low_memory=False)
male1=pd.read_csv("/content/drive/MyDrive/APMLP/Rating_Male.csv", low_memory=False)


In [4]:
fem1 = fem1.groupby('Filename')['Rating'].mean().reset_index()
male1 = male1.groupby('Filename')['Rating'].mean().reset_index()

In [8]:
fem1['numeric_part'] = fem1['Filename'].str.extract('(\d+)').astype(int)
male1['numeric_part'] = male1['Filename'].str.extract('(\d+)').astype(int)
fem1.sort_values(by="numeric_part",inplace=True)
male1.sort_values(by="numeric_part",inplace=True)
fem1 = fem1.set_index('numeric_part')
fem1.index.name =None
male1 = male1.set_index('numeric_part')
male1.index.name =None

In [9]:
fem1

,Filename,Rating
1,AF1.jpg,2.333333
2,AF2.jpg,3.950000
3,AF3.jpg,3.183333
4,AF4.jpg,2.683333
5,AF5.jpg,2.000000
...,...,...
1996,AF1996.jpg,1.600000
1997,AF1997.jpg,1.433333
1998,AF1998.jpg,2.733333
1999,AF1999.jpg,2.733333


In [ ]:
fem1=fem1.drop("original Rating",axis=1)
male1=male1.drop("original Rating",axis=1)

In [10]:
fem1

,Filename,Rating
1,AF1.jpg,2.333333
2,AF2.jpg,3.950000
3,AF3.jpg,3.183333
4,AF4.jpg,2.683333
5,AF5.jpg,2.000000
...,...,...
1996,AF1996.jpg,1.600000
1997,AF1997.jpg,1.433333
1998,AF1998.jpg,2.733333
1999,AF1999.jpg,2.733333


In [11]:
male1

,Filename,Rating
1,AM1.jpg,3.950000
2,AM2.jpg,3.600000
3,AM3.jpg,4.050000
4,AM4.jpg,3.316667
5,AM5.jpg,2.300000
...,...,...
1996,AM1996.jpg,3.816667
1997,AM1997.jpg,4.433333
1998,AM1998.jpg,3.250000
1999,AM1999.jpg,3.416667


In [31]:
all1=pd.concat([fem1, male1], ignore_index=True)

In [32]:
all1

,Filename,Rating
0,AF1.jpg,2.333333
1,AF2.jpg,3.950000
2,AF3.jpg,3.183333
3,AF4.jpg,2.683333
4,AF5.jpg,2.000000
...,...,...
3995,AM1996.jpg,3.816667
3996,AM1997.jpg,4.433333
3997,AM1998.jpg,3.250000
3998,AM1999.jpg,3.416667


In [34]:
femfm=fem1.copy()
malefm=male1.copy()
allfm=all1.copy()

In [42]:
femfm["Filepaths"]="/content/drive/MyDrive/APMLP/Data_Images/"+femfm["Filename"]
malefm["Filepaths"]="/content/drive/MyDrive/APMLP/Data_Images/"+malefm["Filename"]
allfm["Filepaths"]="/content/drive/MyDrive/APMLP/Data_Images/"+allfm["Filename"]
femfm['Filepaths'] = femfm['Filepaths'].astype(str)
malefm['Filepaths'] = malefm['Filepaths'].astype(str)
allfm['Filepaths'] = allfm['Filepaths'].astype(str)

In [49]:
femfm=femfm.drop(columns="Filename")
malefm=malefm.drop(columns="Filename")
allfm=allfm.drop(columns="Filename")

In [50]:
femfm

,Rating,Filepaths
1,2.333333,/content/drive/MyDrive/APMLP/Data_Images/AF1.jpg
2,3.950000,/content/drive/MyDrive/APMLP/Data_Images/AF2.jpg
3,3.183333,/content/drive/MyDrive/APMLP/Data_Images/AF3.jpg
4,2.683333,/content/drive/MyDrive/APMLP/Data_Images/AF4.jpg
5,2.000000,/content/drive/MyDrive/APMLP/Data_Images/AF5.jpg
...,...,...
1996,1.600000,/content/drive/MyDrive/APMLP/Data_Images/AF199...
1997,1.433333,/content/drive/MyDrive/APMLP/Data_Images/AF199...
1998,2.733333,/content/drive/MyDrive/APMLP/Data_Images/AF199...
1999,2.733333,/content/drive/MyDrive/APMLP/Data_Images/AF199...


In [51]:
malefm

,Rating,Filepaths
1,3.950000,/content/drive/MyDrive/APMLP/Data_Images/AM1.jpg
2,3.600000,/content/drive/MyDrive/APMLP/Data_Images/AM2.jpg
3,4.050000,/content/drive/MyDrive/APMLP/Data_Images/AM3.jpg
4,3.316667,/content/drive/MyDrive/APMLP/Data_Images/AM4.jpg
5,2.300000,/content/drive/MyDrive/APMLP/Data_Images/AM5.jpg
...,...,...
1996,3.816667,/content/drive/MyDrive/APMLP/Data_Images/AM199...
1997,4.433333,/content/drive/MyDrive/APMLP/Data_Images/AM199...
1998,3.250000,/content/drive/MyDrive/APMLP/Data_Images/AM199...
1999,3.416667,/content/drive/MyDrive/APMLP/Data_Images/AM199...


In [52]:
allfm

,Rating,Filepaths
0,2.333333,/content/drive/MyDrive/APMLP/Data_Images/AF1.jpg
1,3.950000,/content/drive/MyDrive/APMLP/Data_Images/AF2.jpg
2,3.183333,/content/drive/MyDrive/APMLP/Data_Images/AF3.jpg
3,2.683333,/content/drive/MyDrive/APMLP/Data_Images/AF4.jpg
4,2.000000,/content/drive/MyDrive/APMLP/Data_Images/AF5.jpg
...,...,...
3995,3.816667,/content/drive/MyDrive/APMLP/Data_Images/AM199...
3996,4.433333,/content/drive/MyDrive/APMLP/Data_Images/AM199...
3997,3.250000,/content/drive/MyDrive/APMLP/Data_Images/AM199...
3998,3.416667,/content/drive/MyDrive/APMLP/Data_Images/AM199...


In [53]:
alltrain_df, alltest_df = train_test_split(allfm, train_size=0.7, shuffle =True, random_state=1)

In [54]:
alltrain_df

,Rating,Filepaths
1850,2.833333,/content/drive/MyDrive/APMLP/Data_Images/AF185...
2293,2.650000,/content/drive/MyDrive/APMLP/Data_Images/AM294...
576,2.916667,/content/drive/MyDrive/APMLP/Data_Images/AF577...
2731,2.183333,/content/drive/MyDrive/APMLP/Data_Images/AM732...
1051,2.783333,/content/drive/MyDrive/APMLP/Data_Images/AF105...
...,...,...
3839,3.350000,/content/drive/MyDrive/APMLP/Data_Images/AM184...
1096,2.466667,/content/drive/MyDrive/APMLP/Data_Images/AF109...
3980,3.883333,/content/drive/MyDrive/APMLP/Data_Images/AM198...
235,3.900000,/content/drive/MyDrive/APMLP/Data_Images/AF236...


In [57]:
all_train_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)
all_test_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [58]:
all_train_images=all_train_gen.flow_from_dataframe(dataframe=alltrain_df,
                                  x_col="Filepaths",
                                  y_col="Rating",
                                  target_size=(120,120),
                                  color_mode="rgb",
                                  class_mode="raw",
                                  batc_size=32,
                                  shuffle=True,
                                  seed=42,
                                  subset="training")

all_val_images=all_train_gen.flow_from_dataframe(dataframe=alltrain_df,
                                  x_col="Filepaths",
                                  y_col="Rating",
                                  target_size=(120,120),
                                  color_mode="rgb",
                                  class_mode="raw",
                                  batc_size=32,
                                  shuffle=True,
                                  seed=42,
                                  subset="validation")

all_test_images=all_test_gen.flow_from_dataframe(dataframe=alltest_df,
                                  x_col="Filepaths",
                                  y_col="Rating",
                                  target_size=(120,120),
                                  color_mode="rgb",
                                  class_mode="raw",
                                  batc_size=32,
                                  shuffle=False)

Found 2240 validated image filenames.
Found 560 validated image filenames.
Found 1200 validated image filenames.


In [73]:
inputs=tf.keras.Input(shape=(120,120,3))
x=tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation="relu")(inputs)
x=tf.keras.layers.MaxPool2D()(x)
x=tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation="relu")(x)
x=tf.keras.layers.MaxPool2D()(x)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
x=tf.keras.layers.Dense(64, activation="relu")(x)
x=tf.keras.layers.Dense(64, activation="relu")(x)
outputs=tf.keras.layers.Dense(1, activation="linear")(x)
model=tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam",
              loss="mse")

history=model.fit(
    all_train_images,
    validation_data=all_val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
70/70 [==============================] - 696s 10s/step - loss: 2.0352 - val_loss: 0.4613
Epoch 2/100
70/70 [==============================] - 33s 473ms/step - loss: 0.5180 - val_loss: 0.4406
Epoch 3/100
70/70 [==============================] - 33s 470ms/step - loss: 0.4812 - val_loss: 0.4035
Epoch 4/100
70/70 [==============================] - 29s 419ms/step - loss: 0.4467 - val_loss: 0.3854
Epoch 5/100
70/70 [==============================] - 31s 443ms/step - loss: 0.4213 - val_loss: 0.3786
Epoch 6/100
70/70 [==============================] - 32s 448ms/step - loss: 0.4170 - val_loss: 0.3792
Epoch 7/100
70/70 [==============================] - 29s 408ms/step - loss: 0.4116 - val_loss: 0.4013
Epoch 8/100
70/70 [==============================] - 29s 408ms/step - loss: 0.4071 - val_loss: 0.3887
Epoch 9/100
70/70 [==============================] - 31s 435ms/step - loss: 0.4137 - val_loss: 0.3698
Epoch 10/100
70/70 [==============================] - 30s 422ms/step - loss: 0.4151

In [75]:
import numpy as np
all_pred_ratings=np.squeeze(model.predict(all_test_images))
all_true_ratings=alltest_df["Rating"]

rmse=np.sqrt(model.evaluate(all_test_images, verbose=0))
print("Test RMSE: {:.5f}".format(rmse))
r2=r2_score(all_true_ratings, all_pred_ratings)
print("Test R^2 Score: {:.5f}".format(r2))



38/38 [==============================] - 286s 8s/step
Test RMSE: 0.61498
Test R^2 Score: 0.19876


In [61]:
inputs

<KerasTensor: shape=(None, 120, 120, 3) dtype=float32 (created by layer 'input_1')>

In [62]:
x

<KerasTensor: shape=(None, 118, 118, 16) dtype=float32 (created by layer 'conv2d')>

In [64]:
x

<KerasTensor: shape=(None, 59, 59, 16) dtype=float32 (created by layer 'max_pooling2d')>

In [68]:
x

<KerasTensor: shape=(None, 28, 28, 32) dtype=float32 (created by layer 'max_pooling2d_4')>

In [71]:
tf.keras.layers.GlobalAveragePooling2D()(x)

<KerasTensor: shape=(None, 32) dtype=float32 (created by layer 'global_average_pooling2d')>

In [12]:
male1train=male1[:1400]

In [13]:
fem1train=fem1[:1400]

In [14]:
male1train

,Filename,Rating
1,AM1.jpg,3.950000
2,AM2.jpg,3.600000
3,AM3.jpg,4.050000
4,AM4.jpg,3.316667
5,AM5.jpg,2.300000
...,...,...
1396,AM1396.jpg,2.766667
1397,AM1397.jpg,2.850000
1398,AM1398.jpg,2.666667
1399,AM1399.jpg,2.766667


In [15]:
fem1train

,Filename,Rating
1,AF1.jpg,2.333333
2,AF2.jpg,3.950000
3,AF3.jpg,3.183333
4,AF4.jpg,2.683333
5,AF5.jpg,2.000000
...,...,...
1396,AF1396.jpg,4.050000
1397,AF1397.jpg,3.583333
1398,AF1398.jpg,1.366667
1399,AF1399.jpg,3.183333


In [16]:
male1test=male1[1400:]
male1test

,Filename,Rating
1401,AM1401.jpg,2.400000
1402,AM1402.jpg,3.150000
1403,AM1403.jpg,2.816667
1404,AM1404.jpg,2.900000
1405,AM1405.jpg,2.600000
...,...,...
1996,AM1996.jpg,3.816667
1997,AM1997.jpg,4.433333
1998,AM1998.jpg,3.250000
1999,AM1999.jpg,3.416667


In [17]:
fem1test=fem1[1400:]
fem1test

,Filename,Rating
1401,AF1401.jpg,4.233333
1402,AF1402.jpg,3.833333
1403,AF1403.jpg,4.266667
1404,AF1404.jpg,2.850000
1405,AF1405.jpg,4.116667
...,...,...
1996,AF1996.jpg,1.600000
1997,AF1997.jpg,1.433333
1998,AF1998.jpg,2.733333
1999,AF1999.jpg,2.733333


In [28]:
fem1val1=fem1[1401:1700]
fem1val2=fem1[1701:2000]
male1val1=male1[1401:1700]
male1val2=male1[1701:2000]

In [24]:
fem1val1

,Filename,Rating
1402,AF1402.jpg,3.833333
1403,AF1403.jpg,4.266667
1404,AF1404.jpg,2.850000
1405,AF1405.jpg,4.116667
1406,AF1406.jpg,3.766667
...,...,...
1696,AF1696.jpg,4.000000
1697,AF1697.jpg,4.016667
1698,AF1698.jpg,3.250000
1699,AF1699.jpg,3.400000


In [26]:
fem1val2

,Filename,Rating
1702,AF1702.jpg,2.316667
1703,AF1703.jpg,4.016667
1704,AF1704.jpg,2.166667
1705,AF1705.jpg,3.133333
1706,AF1706.jpg,3.500000
...,...,...
1996,AF1996.jpg,1.600000
1997,AF1997.jpg,1.433333
1998,AF1998.jpg,2.733333
1999,AF1999.jpg,2.733333


In [29]:
male1val1

,Filename,Rating
1402,AM1402.jpg,3.150000
1403,AM1403.jpg,2.816667
1404,AM1404.jpg,2.900000
1405,AM1405.jpg,2.600000
1406,AM1406.jpg,2.683333
...,...,...
1696,AM1696.jpg,4.466667
1697,AM1697.jpg,4.216667
1698,AM1698.jpg,4.400000
1699,AM1699.jpg,4.533333


In [30]:
male1val2

,Filename,Rating
1702,AM1702.jpg,3.500000
1703,AM1703.jpg,3.533333
1704,AM1704.jpg,4.400000
1705,AM1705.jpg,3.433333
1706,AM1706.jpg,2.600000
...,...,...
1996,AM1996.jpg,3.816667
1997,AM1997.jpg,4.433333
1998,AM1998.jpg,3.250000
1999,AM1999.jpg,3.416667


In [ ]:
from PIL import Image
import os

def check_image_size(image_path):
    try:
        img = Image.open(image_path)

        size = img.size

        return size

    except Exception as e:
        print(f"Error checking image size for {image_path}: {str(e)}")
        return None

In [ ]:
import os

directory_path = "/content/drive/MyDrive/APMLP/Data_Images"

for filename in os.listdir(directory_path):
    image_path = os.path.join(directory_path, filename)
    image_size = check_image_size(image_path)

    if image_size:
        print(f"The size of the image at {image_path} is: {image_size} pixels.")


The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM1949.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM1942.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM216.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM1945.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM1940.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM1944.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM215.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM194.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM213.jpg is: (350, 350) pixels.
The size of the image at /content/drive/MyDrive/APMLP/Data_Images/AM1937.jpg is: (350, 350) pixels.
The 

In [ ]:
from sklearn.model_selection import train_test_split
X=
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
from PIL import Image
import numpy as np

def normalize_image(image_path):
    img = Image.open(image_path)

    img_array = np.array(img)

    normalized_img = img_array / 255.0

    return normalized_img


In [ ]:
import os
import numpy as np

def normalize_dataset_m(dataset_path,s,p):
    normalized_images = []

    for i in range (s,p):
        filename="/content/drive/MyDrive/APMLP/Data_Images"+"/AM"+str(i+1)+".jpg"
        if filename.endswith(".jpg"):
            image_path = os.path.join(dataset_path, filename)
            normalized_img = normalize_image(image_path)
            normalized_images.append(normalized_img)

    return np.array(normalized_images)

In [ ]:
import os
import numpy as np

def normalize_dataset_f(dataset_path,s,p):
    normalized_images = []

    for i in range (s,p):
        filename="/content/drive/MyDrive/APMLP/Data_Images"+"/AF"+str(i+1)+".jpg"
        if filename.endswith(".jpg"):
            image_path = os.path.join(dataset_path, filename)
            normalized_img = normalize_image(image_path)
            normalized_images.append(normalized_img)

    return np.array(normalized_images)

In [ ]:
import time
import numpy as np

dataset_path = "/content/drive/MyDrive/APMLP/Data_Images"
normalized_dataset_m_1 = normalize_dataset_m(dataset_path,0,500)
np.save("/content/drive/MyDrive/APMLP/normalized_dataset_m_1.npy", normalized_dataset_m_1)
del normalized_dataset_m_1
#time.sleep(60)
normalized_dataset_m_2 = normalize_dataset_m(dataset_path,500,1000)
np.save("/content/drive/MyDrive/APMLP/normalized_dataset_m_2.npy", normalized_dataset_m_2)
del normalized_dataset_m_2
#time.sleep(60)
normalized_dataset_m_3 = normalize_dataset_m(dataset_path,1000,1500)
np.save("/content/drive/MyDrive/APMLP/normalized_dataset_m_3.npy", normalized_dataset_m_3)
del normalized_dataset_m_3
#time.sleep(60)
normalized_dataset_m_4 = normalize_dataset_m(dataset_path,1500,2000)
np.save("/content/drive/MyDrive/APMLP/normalized_dataset_m_4.npy", normalized_dataset_m_4)
del normalized_dataset_m_4
#time.sleep(60)
normalized_dataset_f_1 = normalize_dataset_f(dataset_path,0,500)
np.save("/content/drive/MyDrive/APMLP/normalized_dataset_f_1.npy", normalized_dataset_f_1)
del normalized_dataset_f_1
#time.sleep(60)
normalized_dataset_f_2 = normalize_dataset_f(dataset_path,500,1000)
np.save("/content/drive/MyDrive/APMLP/normalized_dataset_f_2.npy", normalized_dataset_f_2)
del normalized_dataset_f_2
#time.sleep(60)
normalized_dataset_f_3 = normalize_dataset_f(dataset_path,1000,1500)
np.save("/content/drive/MyDrive/APMLP/normalized_dataset_f_3.npy", normalized_dataset_f_3)
del normalized_dataset_f_3
#time.sleep(60)
normalized_dataset_f_4 = normalize_dataset_f(dataset_path,1500,2000)
np.save("/content/drive/MyDrive/APMLP/normalized_dataset_f_4.npy", normalized_dataset_f_4)
del normalized_dataset_f_4
#time.sleep(60)
"""
normalized_dataset_m_1 = np.load("normalized_dataset_m_1.npy")
normalized_dataset_m_2 = np.load("normalized_dataset_m_2.npy")
normalized_dataset_m_3 = np.load("normalized_dataset_m_3.npy")
normalized_dataset_m_4 = np.load("normalized_dataset_m_4.npy")
combined_dataset_1 = np.hstack((normalized_dataset_m_1, normalized_dataset_m_2,normalized_dataset_m_3,normalized_dataset_m_4))
np.save("combined_dataset_1.npy", combined_dataset_1)
del combined_dataset_1
del normalized_dataset_m_1
del normalized_dataset_m_2
del normalized_dataset_m_3
del normalized_dataset_m_4

normalized_dataset_f_1 = np.load("normalized_dataset_f_1.npy")
normalized_dataset_f_2 = np.load("normalized_dataset_f_2.npy")
normalized_dataset_f_3 = np.load("normalized_dataset_f_3.npy")
normalized_dataset_f_4 = np.load("normalized_dataset_f_4.npy")
combined_dataset_2 = np.hstack((normalized_dataset_f_1,normalized_dataset_f_2,normalized_dataset_f_3,normalized_dataset_f_4))
np.save("combined_dataset_2.npy", combined_dataset_2)
del combined_dataset_2
del normalized_dataset_f_1
del normalized_dataset_f_2
del normalized_dataset_f_3
del normalized_dataset_f_4

combined_dataset_1 = np.load("combined_dataset_1.npy")
combined_dataset_2 = np.load("combined_dataset_2.npy")
combined_dataset = np.hstack((combined_dataset_1,combined_dataset_2))
combined_dataset

"""

'\nnormalized_dataset_m_1 = np.load("normalized_dataset_m_1.npy")\nnormalized_dataset_m_2 = np.load("normalized_dataset_m_2.npy")\nnormalized_dataset_m_3 = np.load("normalized_dataset_m_3.npy")\nnormalized_dataset_m_4 = np.load("normalized_dataset_m_4.npy")\ncombined_dataset_1 = np.hstack((normalized_dataset_m_1, normalized_dataset_m_2,normalized_dataset_m_3,normalized_dataset_m_4))\nnp.save("combined_dataset_1.npy", combined_dataset_1)\ndel combined_dataset_1\ndel normalized_dataset_m_1\ndel normalized_dataset_m_2\ndel normalized_dataset_m_3\ndel normalized_dataset_m_4\n\nnormalized_dataset_f_1 = np.load("normalized_dataset_f_1.npy")\nnormalized_dataset_f_2 = np.load("normalized_dataset_f_2.npy")\nnormalized_dataset_f_3 = np.load("normalized_dataset_f_3.npy")\nnormalized_dataset_f_4 = np.load("normalized_dataset_f_4.npy")\ncombined_dataset_2 = np.hstack((normalized_dataset_f_1,normalized_dataset_f_2,normalized_dataset_f_3,normalized_dataset_f_4))\nnp.save("combined_dataset_2.npy", com